In [1]:
import cPickle
import numpy as np
import pandas as pd
import os
import re
from collections import Counter, defaultdict
import operator

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
# brands_to_ignore = [] 
# with open('./polysemy_brands.txt') as f:
#     for i in f: brands_to_ignore.append(i.strip('\n'))

# total_recall_summary_data_path = 'total_recall_summary_data'
# cat_dict = {}   ## A dict to save recall data : {'Greek_yogurt': {'Activia': 0.0288808664259928, 'Chobani': 0.18411552346570398,...}, {'Fast_food':...
# brands_name_dict = {}
# # print(os.listdir(total_recall_summary_data_path))
# for p in os.listdir(total_recall_summary_data_path)[1:]:
#     df = pd.read_csv(total_recall_summary_data_path+'/'+p,index_col=0)
#     df = df[['Item_name','Recall_prop']].set_index('Item_name')
#     cat = p.replace('total_recall_rate_','').replace('.csv','')
#     cat_dict[cat] = df.to_dict()['Recall_prop']
#     brands_name_dict[cat] = df.index.tolist()
    

In [3]:
raw_data_path = "data/semantic_fluency_recall_11Apr2018.csv"
raw_data = pd.read_csv(raw_data_path, index_col="ResponseId") #   "Response ID", skiprows=[0])
raw_data.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q27,Q29_1,Q31_1,Q32_1,Q33_1,Q34_1,Q35_1,Q36_1,Q37_1,Q38_1,Q39_1,Q40_1,Q41_1,Q42_1,Q43_1,Q44_1,Q45_1,Q46_1,Q47_1,Q48_1,Q49_1,Q70_1,Q71_1,Q72_1,Q73_1,Q74_1,Q75_1,Q76_1,Q77_1,Q78_1,Q79_1,Q80_1,Q81_1,Q82_1,Q83_1,Q84_1,Q85_1,Q86_1,Q87_1,Q88_1,Q89_1,Q50_1,Q51_1,Q52_1,Q53_1,Q54_1,Q55_1,Q56_1,Q57_1,Q58_1,Q59_1,Q60_1,Q61_1,Q62_1,Q63_1,Q64_1,Q65_1,Q66_1,Q67_1,Q68_1,Q69_1,Q73,Q75,Q79,MTurkCode,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86
ResponseId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Response ID,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Recipient Last Name,Recipient First Name,Recipient Email,External Data Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,Q27,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different kinds ...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different brands...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try to recall any many different fast f...,Please try t

In [4]:
Q_columns = [x for x in raw_data.columns if x[0]=="Q"]
raw_data = raw_data[Q_columns]
raw_data_T = raw_data.T
raw_data_T

ResponseId,Response ID,"{""ImportId"":""_recordId""}",R_p0PhFqO56odKn6N,R_2XoE9PDq0UJv2H1,R_2DSoXYsbtpqebto,R_dhfPdOQdVBd0c9j,R_2ROC575mrjvfKjh,R_z8Uht9EVAzllAxb,R_2EGi1cKjOGOWXeJ,R_2ydsx6RVFS48dv9,R_2WZ52ZRuQTnky7W,R_31jdY4ygRPjq75B,R_2t5LsLfTQO2Ur8u,R_cOSOGijFyc5tdu1,R_1jdCQwi6L98nN0M,R_2uvUD37cLQzxUVJ,R_2f2Tu0XoIAo9Nr3,R_R1SLaOuiepitBG9,R_1pRHSK0SlF1mvhg,R_1rqAFRRo066ddTq,R_Tcv5ALbOn69okSJ,R_21j6hVs5SC89v4K,R_3izh58aAqn0qhol,R_3e2j7xqRGgDQLUv,R_2QFYPkvgOs7Zhln,R_1CgA5LiGa7Xsebq,R_R4sL7HdH4CBG1aN,R_3FV5JZExo2ZFC2J,R_OIqltiXKPLkKvxT,nan,nan,nan,nan,nan,nan
Q27,Q27,"{""ImportId"":""QID27""}",Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,Accept,NaN,NaN,NaN,NaN,NaN,NaN
Q29_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID29_1""}",banana,banana,watermelon,apple,Cherry,apple,apple,apple,grapefruit,apple,apples,apple,bananas,strawberry,apple,Mango,Apple,kiwi,banana,apple,apple,apple,apple,starfruit,Apple,apple,Apple,orange,NaN,strawberry,apple,orange,apple
Q31_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID31_1""}",orange,apple,strawberries,plum,Strawberry,banana,tangerine,pear,pineapple,banana,banana,orange,tomatoes,banana,grapes,Orange,Oranges,orange,apple,banana,orange,pear,mango,apple,Strawberry,banana,Pear,apple,NaN,grape,pineapple,apple,strawberry
Q32_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID32_1""}",lemon,orange,grapes,pear,grape,orenge,ear,raspberry,apple,orange,pear,lemon,cherries,blueberry,oranges,Goa,Grapes,lemon,orange,kiwi,banana,blueberry,kiwi,strawberries,Blueberry,orange,Plum,banana,NaN,tomato,strawberry,grape,pear
Q33_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID33_1""}",apple,grape,kiwi,watermelon,tomato,kiwi,pear,plum,orange,kiwi,peach,grapes,apples,raspberry,cherries,Promogranate,Bananas,cumquat,cherry,orange,pear,grapefruit,orange,banana,Peach,lemon,Grape,kiwi,NaN,rasberry,blueberry,apricot,banana
Q34_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID34_1""}",melons,grapefruit,banana,orange,plumb,NaN,pineapple,peach,pear,blueberry,strawberry,strawberry,peaches,blackberry,blueberries,Watermelon,Grapefruit,pomegranate,kiwi,lemon,watermelon,grape,banana,black berries,Banana,NaN,Peach,pear,NaN,banana,raspberry,strawberry,cherry
Q35_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID35_1""}",kiwi,NaN,NaN,kiwi,nectarine,NaN,cranberry,strawberry,peach,strawberry,watermelon,cherry,grapes,gooseberry,banana,Apple,Strawberries,banana,pear,lime,cantaloupe,banana,pomegranate,raspebeeries,Kiwi,NaN,Tomato,peach,NaN,NaN,orange,bananna,mango
Q36_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID36_1""}",pineapple,NaN,NaN,mango,peach,NaN,blackberry,blackberry,apricot,raspberry,kiwi,rasberry,oranges,apricot,watermelon,NaN,Raspberries,dragonfruit,grape,watermelon,honeydew,watermelon,peach,lingon berries,Starfruit,NaN,Pineapple,mango,NaN,NaN,tangerine,blueberry,pineapple
Q37_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID37_1""}",mango,NaN,NaN,pear,banana,NaN,raspberry,bluberry,banana,blackberry,orange,blueberry,kiwi,apple,canalope,NaN,Blueberries,starfruit,strawberry,cantaloupe,grapes,canteloupe,apricot,boysen berries,Grapes,NaN,Kiwi,strawberry,NaN,NaN,manderin,blackberry,grape
Q38_1,Please try to recall any many different kinds ...,"{""ImportId"":""QID38_1""}",NaN,NaN,NaN,bananna,kiwi,NaN,strawberry,termelon,cherry,boysenberry,tangerine,anana,strawberries,cranberry,pineapple,NaN,Cantaloupe,watermelon,lemon,pineapple,cherries,honeydew melon,nectarine,lychee,Raspberry,NaN,Orange,raspberry,NaN,NaN,pear,raspberry,blackberry


In [5]:
class BrandRecognizer():
    def __init__(self, category):
        self.category = category
        self.brands = brands_name_dict[category]
    
    def subset_with_sequence(self, subname_list, name_list, resilience, min_endurance=2):
        if len(subname_list) == 0:
            return resilience
        else:
            w = subname_list[0] 
            if w in list(name_list):
                new_name_list = name_list[name_list.index(w):]
#                 print(subname_list)
#                 print(new_name_list)
                new_subname_list = subname_list[1:]
            
                if len(new_subname_list) > 0:
                    return self.subset_with_sequence(new_subname_list, new_name_list,resilience)
                else:
                    return resilience
            else:
                resilience+=1
                if resilience>min_endurance:
                    return resilience
                else:
                    return self.subset_with_sequence(subname_list[1:], name_list, resilience)
                    
    def correction(self, brand):
        if not pd.isnull(brand):
            brand_sub = re.sub(r'[^\w\s]','',brand.lower().replace(' ',''))
#             print(brand_sub)
            if len(brand_sub) == 0:
                return brand
            for official_brand in self.brands:
                official_brand_sub = re.sub(r'[^\w\s]','',official_brand.lower().replace(' ',''))

                ## method 1
                if brand_sub in official_brand_sub:
#                     print("method 1")
                    return official_brand

                ## method 2
                elif official_brand_sub in self.edits1(brand_sub):
#                     print("method 2")
                    return official_brand

            ## method 3
#             print("method 3")
            agg_dict = defaultdict(list)
            for official_brand in self.brands:
                official_brand_sub = re.sub(r'[^\w\s]','',official_brand.lower().replace(' ',''))
                resilience_updated = self.subset_with_sequence(list(brand_sub),list(official_brand_sub),resilience=0)
                agg_dict[resilience_updated].append(official_brand)
            if len(agg_dict.keys())>1:
                min_corrected = min(agg_dict.keys())
                if len(agg_dict[min_corrected])==1:
                    return agg_dict[min_corrected][0]
                else:
                    print "{} : {}".format(brand, agg_dict[min_corrected])
                    return brand
            else:
                print brand
                return brand
        else:
            return brand

    def edits1(self,word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits_1 = [word[:i]    for i in range(len(word) + 1)]
        splits_2 = [word[i:]    for i in range(len(word) + 1)]
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(splits_1 + splits_2 + deletes + transposes + replaces + inserts)

In [6]:
demographic_df = raw_data_T[raw_data_T.apply(lambda row: ("age" in row['Response ID']) or \
                                                         ("gender" in row['Response ID']), axis=1)]
# demographic_df.columns = ["Response ID", "ImportId"] + range(demographic_df.shape[1]-2)
demographic_df

ResponseId,Response ID,"{""ImportId"":""_recordId""}",R_p0PhFqO56odKn6N,R_2XoE9PDq0UJv2H1,R_2DSoXYsbtpqebto,R_dhfPdOQdVBd0c9j,R_2ROC575mrjvfKjh,R_z8Uht9EVAzllAxb,R_2EGi1cKjOGOWXeJ,R_2ydsx6RVFS48dv9,R_2WZ52ZRuQTnky7W,R_31jdY4ygRPjq75B,R_2t5LsLfTQO2Ur8u,R_cOSOGijFyc5tdu1,R_1jdCQwi6L98nN0M,R_2uvUD37cLQzxUVJ,R_2f2Tu0XoIAo9Nr3,R_R1SLaOuiepitBG9,R_1pRHSK0SlF1mvhg,R_1rqAFRRo066ddTq,R_Tcv5ALbOn69okSJ,R_21j6hVs5SC89v4K,R_3izh58aAqn0qhol,R_3e2j7xqRGgDQLUv,R_2QFYPkvgOs7Zhln,R_1CgA5LiGa7Xsebq,R_R4sL7HdH4CBG1aN,R_3FV5JZExo2ZFC2J,R_OIqltiXKPLkKvxT,nan,nan,nan,nan,nan,nan
Q73,What is your age?,"{""ImportId"":""QID90_TEXT""}",37,30,24,46,63,31,26,62,32,30,53,39,35,31,65,23,29,28,31,29,52,24,33,29,32,30,62,40,NaN,23,NaN,20,46
Q75,What is your gender?,"{""ImportId"":""QID91""}",Male,Female,Female,Female,Female,Male,Male,Female,Female,Male,Female,Female,Male,Female,Female,Female,Female,Female,Male,Female,Female,Male,Female,Female,Female,Female,Female,1,NaN,2,NaN,1,1


In [7]:
def extract_raw_category(raw_data_T, category, match_term, demographic_numbers=2, save=False):
    ## extract data about one category from the mturk output
    ##
    ## raw_data_T : raw mturk output
    ## category : category name
    ## match_term : category official name at mturk question. eg. "fast_food" -> "fast food"
    ## demographic_numbers : numbers of demographic features, assume 2 for now, includes 'age' and 'gender'
    ## save : save to 'data/raw/{}.csv'.format(category)
    
    cat_list = brands_name_dict[category]
    cat_df = raw_data_T[raw_data_T.apply(lambda row: (match_term in row['Response ID']) or \
                                                          ("age" in row['Response ID']) or \
                                                          ("gender" in row['Response ID'])\
                                              , axis=1)]
    cat_df.index = cat_df.index[:-demographic_numbers].tolist() + ['age','gender']
    print(cat_df.shape)
    cat_df = cat_df.dropna(axis=1, thresh=demographic_numbers+2)
    print(cat_df.shape)
    cat_df.columns = ["Response ID", "ImportId"] + range(cat_df.shape[1]-2)
    if save:
        cat_df.to_csv('data/raw/{}.csv'.format(category))
    return cat_df  
    
## Note now assume only two demographic features : age and gender, if changed, need to change the function
category = "fast_food"
demographic_numbers = 2
fastfood_df = extract_raw_category(raw_data_T, category, match_term="fast food", demographic_numbers=2, save=True)

NameError: global name 'brands_name_dict' is not defined

In [ ]:
def clean_extract_raw_category(cat_df, category, demographic_numbers=2, save=False):
    BR = BrandRecognizer(category)
    cat_df_clean = cat_df.iloc[:-demographic_numbers,2:].applymap(lambda brand : BR.correction(brand))
    # fastfood_df_new = fastfood_df_new.reset_index()
    demo_df = cat_df.iloc[-demographic_numbers:,2:]
    cat_df_clean = pd.concat([cat_df_clean, demo_df])
    if save:
        cat_df_clean.to_csv('data/raw_clean/{}.csv'.format(category))
    return cat_df_clean

fastfood_df_clean = clean_extract_raw_category(fastfood_df, category, demographic_numbers=2, save=True)

In [ ]:
# fastfood_df_new

In [ ]:
# fastfood_df.iloc[:,2:]

In [ ]:
##########################################
## Note now assume only two demographic features : age and gender, if changed, need to change the function
category = "running_shoes"
demographic_numbers = 2
running_shoes_df = extract_raw_category(raw_data_T, category, match_term="shoe", demographic_numbers=2, save=True)

In [ ]:
running_shoes_df_clean = clean_extract_raw_category(running_shoes_df, category, demographic_numbers=2, save=True)

In [ ]:
# BR = BrandRecognizer("fast_food")
# # BR.subset('bk')
# BR.correction("wendys")
# # MAA = BR.edits1("MacDonalds".lower())
# # MAA
# # re.sub(r'[^\w\s]','',"McDonald's".lower().replace(' ',''))